# RAG Application

![Simple RAG](../../images/simple_rag.png)

- LangSmith 문서를 기반으로 응답하는 RAG 구현 테스트

### 환경설정

OpenAI API key와 LANGCHAIN API key를 프로젝트 루트에 .env 환경변수 파일 생성 후 입력하기

In [1]:
# 환경변수 로드
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Simple RAG application

In [2]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

""" 
retrieve_documents
- 사용자의 질문을 기반으로 벡터스토어에서 가져온 문서를 반환합니다.(* langSmith 문서 기반)
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response  
- 입력을 포맷팅한 후 `call_openai`를 호출하여 모델 응답을 생성합니다.
"""
@traceable(run_type="chain")
def generate_response(question: str, documents): # 질문과 검색된 문서 인자로 받음.
    formatted_docs = "\n\n".join(doc.page_content for doc in documents) # 검색된 문서 덩어리를 하나의 문서로 변환(포맷팅)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT # 시스템 프롬프트 입력
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}" # 검색된 문서를 Context로 사용자의 질문을 Question으로 전달
        }
    ]
    return call_openai(messages) # LLM 모델 호출 함수에 메시지 전달(시스템, 유저) 및 응답 결과 반환

"""
call_openai  
- OpenAI의 채팅 완성 결과를 반환합니다.
"""
@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

"""
langsmith_rag
1. `retrieve_documents`를 호출하여 문서를 가져옵니다.
2. 가져온 문서를 기반으로 `generate_response`를 호출하여 응답을 생성합니다.
3. 모델 응답을 반환합니다.
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):  # -> 사용자 질문(question)을 인자로 전달
    documents = retrieve_documents(question)  # 1. 질문을 기반으로 벡터스토어에서 관련 문서를 검색
    response = generate_response(question, documents)  # 2. 질문과 검색된 문서를 기반으로 모델 응답 생성
    return response.choices[0].message.content  # 3. 생성된 응답의 콘텐츠 반환


USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 219/219 [00:07<00:00, 30.47it/s]


This should take a little less than a minute. We are indexing and storing LangSmith documentation in a SKLearn vector database.

In [3]:
question = "LangSmith는 무엇에 사용되나요?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)

LangSmith는 LLM 애플리케이션 개발, 모니터링 및 테스트를 위한 플랫폼입니다. 이 플랫폼은 프로토타입 제작, 디버깅 및 애플리케이션 개발 생애 주기의 각 단계에서 다양한 워크플로우를 지원합니다. 사용자는 LangSmith를 통해 모델 성능을 이해하고 문제를 해결할 수 있습니다.


### LangSmith 결과 확인

<img width="1081" alt="image" src="https://github.com/user-attachments/assets/30bf261f-5ec4-4d74-b359-e42f67bc64c9" />